In [10]:
import ROOT
from ROOT import RDataFrame, TCanvas, THStack
%jsroot on

In [11]:
xsec_info = {
    "ttbar": 396.87 + 332.97, # nonallhad + allhad, keep same x-sec for all
    "single_top_s_chan": 2.0268 + 1.2676,
    "single_top_t_chan": (36.993 + 22.175)/0.252,  # scale from lepton filter to inclusive
    "single_top_tW": 37.936 + 37.906,
    "wjets": 61457 * 0.252,  # e/mu+nu final states
    "data": None
}

def get_histo(source, nbins=25, bin_low=50, bin_high=550):
    
    d = RDataFrame('events', source+'.root')

    x_sec = xsec_info[source]
    nevts_total = d.Count().GetValue()
    lumi = 3378 # /pb
    xsec_weight = x_sec * lumi / nevts_total
    
    
    d = d.Define('electron_pt_mask', 'electron_pt>25').Define('muon_pt_mask', 'muon_pt>25').Define('jet_pt_mask', 'jet_pt>25')\
         .Filter('Sum(electron_pt_mask) + Sum(muon_pt_mask) == 1')\
         .Filter('Sum(jet_pt_mask) >= 4')\
         .Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>=2')            


    jet_p4 = d.Define("jet_p4", 
        "ROOT::VecOps::Construct<ROOT::Math::PxPyPzMVector>(jet_px[jet_pt_mask], jet_py[jet_pt_mask], jet_pz[jet_pt_mask], jet_mass[jet_pt_mask])"
    )

    trijet = jet_p4.Define('trijet', 
        'ROOT::VecOps::Combinations(jet_pt[jet_pt_mask],3)'
    ).Define('ntrijet', 'trijet[0].size()')

    trijet_p4 = trijet.Define('trijet_p4', 
                          'ROOT::VecOps::RVec<ROOT::Math::PxPyPzMVector> trijet_p4(ntrijet);'              +\
                          'for (int i = 0; i < ntrijet; ++i) {'                                            +\
                              'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'       +\
                              'trijet_p4[i] = jet_p4[j1] + jet_p4[j2] + jet_p4[j3];'                       +\
                          '}'                                                                              +\
                          'return trijet_p4;'                                                                                                                          
                         )

    #TODO  implement references
    trijet_pt = trijet_p4.Define('trijet_pt', 
            'return (ROOT::VecOps::Map(trijet_p4, [](ROOT::Math::PxPyPzMVector v) { return v.Pt(); }))'
                                )

    trijet_pt_btag = trijet_pt.Define('trijet_btag', 
                                      'ROOT::VecOps::RVec<bool> btag(ntrijet);'                                   +\
                                      'for (int i = 0; i < ntrijet; ++i) {'                                       +\
                                       'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'     +\
                                       'btag[i]=std::max({jet_btag[j1], jet_btag[j2], jet_btag[j3]})>0.5;'        +\
                                      '}'                                                                         +\
                                      'return btag;'
                                )

    trijet_mass=trijet_pt_btag.Define('trijet_mass',
                                      'double mass;'+\
                                      'double Pt = 0;'+\
                                      'double indx = 0;'+\
                                      'for (int i = 0; i < ntrijet; ++i) {'               +\
                                      '    if ((Pt < trijet_pt[i]) && (trijet_btag[i])) {'+\
                                      '        Pt = trijet_pt[i];'+\
                                      '        indx=i;'+\
                                      '    }'                                            +\
                                      '}'                                                +\
                                      'mass = trijet_p4[indx].M();'             +\
                                      'return mass;'
                                     )

    
    mass = trijet_mass.Define('weights', str(xsec_weight)).Histo1D(('h_'+source, source, nbins, bin_low, bin_high), 'trijet_mass', 'weights')
#     mass.Scale(xsec_weight)
    print(source +' histogram has been created')
    return mass
    

In [12]:
import json
with open('ntuples.json') as f:
    processes = list(json.load(f).keys())
processes.remove('data')
processes

['ttbar', 'single_top_s_chan', 'single_top_t_chan', 'single_top_tW', 'wjets']

In [21]:
ttbar = get_histo(processes[0])
ttbar.SetFillColor(ROOT.kOrange)
# st.Add(ttbar.GetPtr())
# ttbar.Draw('hist')

top_s_chan = get_histo(processes[1])
top_s_chan.SetFillColor(ROOT.kBlack)
# st.Add(top_s_chan.GetPtr())
# top_s_chan.Draw('same&hist')

top_tW = get_histo(processes[3])
top_tW.SetFillColor(ROOT.kBlue)
# st.Add(top_tW.GetPtr())
# top_tW.Draw('same&hist')

top_t_chan = get_histo(processes[2])
top_t_chan.SetFillColor(ROOT.kViolet)
# st.Add(top_t_chan.GetPtr())
# top_s_chan.Draw('same&hist')

wjets = get_histo(processes[4])
wjets.SetFillColor(ROOT.kRed)
# st.Add(wjets.GetPtr())
# wjets.Draw('same&hist')

# c.Draw()


ttbar histogram has been created
single_top_s_chan histogram has been created
single_top_tW histogram has been created
single_top_t_chan histogram has been created
wjets histogram has been created


In [22]:
c = TCanvas('c', 'c', 600, 500)
st = THStack()
st.Add(top_t_chan.GetPtr())
st.Add(wjets.GetPtr())
st.Add(top_tW.GetPtr())
st.Add(top_s_chan.GetPtr())
st.Add(ttbar.GetPtr())
st.Draw('stack hist')
st.SetTitle('RDF t#bar{t}-analysis')
c.BuildLegend(0.7, 0.6, 0.9, 0.9)
c.Draw()
c.SaveAs('rdf_analysis.pdf')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file rdf_analysis.pdf has been created


In [15]:
tterrorbar = ttbar
c1 = TCanvas()
tterrorbar.Draw('hist')
tterrorbar.Draw('same&E0')
tterrorbar.GetXaxis().SetTitle("#m_{bjj} [Gev]")
tterrorbar.SetTitle("Jet energy variations");
tterrorbar.SetFillColor(ROOT.kWhite)
c1.Draw()
c1.SaveAs('rdf_jetvar.png')

Info in <TCanvas::Print>: png file rdf_jetvar.png has been created
